In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [2]:
def load_and_preprocess_images(data_dir):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        for image_file in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_file)
            image = Image.open(image_path).convert('L')  # Convert to grayscale
            image = image.resize((28, 28))  # Resize to 28x28 pixels
            image = np.array(image) / 255.0  # Normalize to [0, 1]
            images.append(image)
            labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

data_dir = "C:/Users/Sudha/Downloads/alphabets_dataset"

# Load and preprocess the data
images, labels = load_and_preprocess_images(data_dir)

# Reshape images for the CNN input
images = images.reshape(-1, 28, 28, 1)

# Convert labels to categorical format
labels = to_categorical(labels)

# Check the shape of the data
print(f'Images shape: {images.shape}')
print(f'Labels shape: {labels.shape}')


KeyboardInterrupt: 

In [ ]:
def visualize_images(images, labels, num_samples=5):
    plt.figure(figsize=(10, 2))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i+1)
        plt.imshow(images[i], cmap='gray')
        plt.title(f'Label: {labels[i]}')
        plt.axis('off')
    plt.show()

# Visualize the first 5 images
visualize_images(images, labels)

In [ ]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

In [ ]:
model.save('handwritten_ocr_model.h5')

In [ ]:
def predict_text(image_path, model):
    image = Image.open(image_path).convert('L')
    image = image.resize((28, 28))
    image = np.array(image) / 255.0
    image = image.reshape(1, 28, 28, 1)
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction, axis=1)
    return predicted_label

# Load the trained model
from tensorflow.keras.models import load_model

model = load_model('handwritten_ocr_model.h5')

# Predict a single image
image_path = 'C:/Users/Sudha/Downloads/alphabets_dataset/alphabet_images/image_12936.png'
predicted_label = predict_text(image_path, model)
print(f'Predicted Label: {predicted_label}')